In [1]:
# s2c9 Implement PKCS#7 padding
#from Set1 import *
def pad(b: bytes, block_size: int=16, protocol: str='pkcs7') -> bytes:
    lb = len(b)
    n_blocks, rem = divmod(lb, block_size)
    if not rem:
        return b
    d = block_size-rem
    return b+bytes(d for _ in range(d))


pad(b'YELLOW SUBMARINE', 20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

In [2]:
# These are some useful simple and flexible reference implementations of the modes.
from typing import Callable, Generator
from functools import reduce
import operator

# ToDo: Learn how to import in notebooks
#import itertools
#def xor_bytes(b1: bytes, b2: bytes, padding: int=0) -> bytes:
#    return bytes(i^j for i,j in itertools.zip_longest(b1, b2, fillvalue=padding))


def xor_bytes(b1: bytes, b2: bytes, padding: int = 0) -> bytes:
    #assert len(b1) == len(b2)
    return bytes(i ^ j for i, j in zip(b1, b2))


def blocks(l: bytes, n: int) -> Generator:  # ToDo: Learn how to declare a generator on mypy 
    for i in range(0, len(l), n):
        yield l[i:i + n]


def ecb(plaintext: bytes, key: bytes, f: Callable = xor_bytes) -> Generator:
    for block in blocks(plaintext, len(key)):
        yield f(block, key)


def cbc_forward(plaintext: bytes, key: bytes, iv: bytes = None, f: Callable = xor_bytes) -> Generator:
    if not iv:
        iv = b'0' * len(key)
    for block in blocks(plaintext, len(key)):
        c = f(f(block, iv), key)
        yield c
        iv = c


def cbc_backwards(plaintext: bytes, key: bytes, iv: bytes = None, f: Callable = xor_bytes) -> Generator:
    if not iv:
        iv = b'0' * len(key)
    for block in blocks(plaintext, len(key)):
        c = f(f(block, key), iv)
        yield c
        iv = block


def crypt(plaintext: bytes, key: bytes, mode: Callable = ecb):
    return reduce(operator.add, mode(plaintext, key))

key = b'YELLOW SUBMARINE'
print(crypt(crypt(b'Testing encryption and decryption with ECB', key, mode=ecb), key, mode=ecb))
print(crypt(crypt(b'Testing encryption and decryption with CBC', key, mode=cbc_forward), key, mode=cbc_backwards))

b'Testing encryption and decryption with ECB'
b'Testing encryption and decryption with CBC'


In [3]:
# s2c10 Implement CBC mode
import base64
import functools
try:
    import Crypto
except:
    import crypto
    import sys
    sys.modules['Crypto'] = crypto

from Crypto.Cipher import AES


def aes_encrypt(block, key):
    aes = AES.new(key, AES.MODE_ECB)
    return aes.encrypt(block)
aes_f = functools.partial(cbc_forward, f=aes_encrypt)


def aes_cbc_forward(plaintext: bytes, key: bytes, iv: bytes=None) -> Generator:
    if not iv:
        iv = b'0'*len(key)
    aes = AES.new(key, AES.MODE_ECB)
    for block in blocks(plaintext, len(key)):
        c = aes.encrypt(xor_bytes(block, iv))
        yield c
        iv = c

        
def aes_cbc_backwards(plaintext: bytes, key: bytes, iv: bytes=None) -> Generator:
    if not iv:
        iv = b'0'*len(key)
    aes = AES.new(key, AES.MODE_ECB)
    for block in blocks(plaintext, len(key)):
        c = xor_bytes(aes.decrypt(block), iv)
        yield c
        iv = block

key = b'YELLOW SUBMARINE'
crypt(crypt(pad(b'Testing encryption and decryption with CBC'), key, mode=aes_cbc_forward), key, mode=aes_cbc_backwards)

cipher = base64.b64decode(open('10.txt').read())

#test
aes = AES.new(b'YELLOW SUBMARINE', AES.MODE_CBC, b'0'*16)
assert aes.decrypt(cipher) == crypt(cipher, b'YELLOW SUBMARINE', mode=aes_cbc_backwards)

crypt(cipher, b'YELLOW SUBMARINE', mode=aes_cbc_backwards)

b"y\x17]\x10RQS[\x10Q^T\x10y\x17] ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can be

In [4]:
# s2c11
# An ECB/CBC detection oracle
from typing import Tuple
from statistics import stdev
from collections import Counter

import secrets
secrets.token_bytes(16)  # Generate a random key


def random_crypt(plaintext: bytes) -> Tuple[bytes, int]:
    new_plain = pad(secrets.token_bytes(secrets.choice(range(10))+1) + plaintext + secrets.token_bytes(secrets.choice(range(10))+1))
    key = secrets.token_bytes(16)
    args = secrets.choice([(key, AES.MODE_ECB), (key, AES.MODE_CBC, secrets.token_bytes(16))])
    aes = AES.new(*args)
    return aes.encrypt(new_plain), args[1]


def mode_oracle(blackbox: Callable):
    ciphertext, used_mode = blackbox(b'A'*1000)
    if stdev(Counter(ciphertext).values()) > 5:
        mode = AES.MODE_ECB
    else:
        mode = AES.MODE_CBC
    if used_mode == mode:
        print('Correctly guessed mode')
    else:
        print('Incorrectly guessed')
    return 'ECB' if mode == AES.MODE_ECB else 'CBC'
    
mode_oracle(random_crypt)

Correctly guessed mode


'CBC'

In [5]:
random_key = secrets.token_bytes(16)

In [23]:
# s2c12
import base64


def generate_encrypter(key: bytes) -> Callable:
    def consistent_crypt(plaintext: bytes) -> bytes:
        unknown = b'Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK'
        new_plain = pad(plaintext + base64.b64decode(unknown))
        aes = AES.new(key, AES.MODE_ECB)
        return aes.encrypt(new_plain)
    return consistent_crypt


def guess_block_size(blackbox: Callable) -> int:
    for i in range(1, 256):
        c = blackbox(b'A'*i*2)
        if c[:i] == c[i:2*i]:
            return i


def guess_block(blackbox: Callable, block_number: int, b_size: int, prev_block: bytes=b'') -> bytes:
    found = b''
    while len(found) < b_size:
        base = b'A'*(b_size-len(found)-1)
        known = blackbox(base)
        for i in range(256):
            plain = base + prev_block + bytes(found) + bytes([i])
            cipher = blackbox(plain)
            bracket = b_size*block_number
            if cipher[bracket:bracket+b_size] == known[bracket:bracket+b_size]:
                found += bytes([i])
                break
        else:
            # Didn't find any other matching byte
            return found
    return found


def guess_append(blackbox: Callable):
    b_size = guess_block_size(blackbox)
    mode = mode_oracle(lambda x: (blackbox(x), 1))
    if mode != 'ECB':
        raise Exception("Can't crack anything other than ECB")

    found, new_block = b'', b'placeholder'
    block = 0
    while new_block:
        new_block = guess_block(blackbox, block, b_size, found)
        found += new_block
        block += 1        
    return found

blackbox = generate_encrypter(random_key)
print(guess_append(blackbox))

Correctly guessed mode
b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x01"
